# Execute PCP generic pipeline

In [2]:
from pathlib import Path

from pipecraft.backend.snakemake import SnakeMakeBackend, SnakeMakeConfig

from starrynight.experiments.common import AcquisitionOrderType, ImageFrameType
from starrynight.experiments.pcp_generic import PCPGeneric, PCPGenericInitConfig
from starrynight.modules.analysis.analysis_cp import AnalysisInvokeCPModule
from starrynight.modules.analysis.analysis_cppipe import AnalysisGenCPPipeModule

# inventory and index
from starrynight.modules.gen_index import GenIndexModule
from starrynight.modules.gen_inv import GenInvModule
from starrynight.pipelines.pcp_generic import create_pcp_generic_pipeline
from starrynight.schema import DataConfig

## Setup dataset paths

In [3]:
dataset_path = Path("../../scratch/starrynight_example_input")
workspace_path = Path("../../scratch/starrynight_example_output")
exec_runs = Path("../../scratch/starrynight_runs_full")
exec_mounts = Path("../../scratch/starrynight_mounts")

## Create data config

In [5]:
data_config = DataConfig(
    dataset_path=dataset_path,
    storage_path=dataset_path,
    workspace_path=workspace_path,
)

## Create execution engine config config

In [5]:
backend_config = SnakeMakeConfig(
    use_fluent_bit=False, print_exec=True, background=False
)

## Configure the generate inventory module
This module is special and doesn't require an experiment for configuration

In [6]:
gen_inv_mod = GenInvModule.from_config(data_config)
exec_backend = SnakeMakeBackend(
    gen_inv_mod.pipe, backend_config, exec_runs / "run001", exec_mounts
)
run = exec_backend.run()
run.wait()

Assuming unrestricted shared filesystem usage.
host: GPFDA-11A
Building DAG of jobs...
Using shell: /nix/store/8vpg72ik2kgxfj05lc56hkqrdrfl8xi9-bash-5.2p37/bin/bash
Provided cores: 10
Rules claiming more threads will be scaled down.
Singularity containers: ignored
Job stats:
job                   count
------------------  -------
all                       1
generate_inventory        1
total                     2

Select jobs to execute...
Execute 1 jobs...

[Thu Apr 17 17:36:52 2025]
localrule generate_inventory:
    output: /home/ank/workspace/hub/broad/starrynight/scratch/starrynight_example_output/inventory/completed.txt
    jobid: 1
    reason: Forced execution
    resources: tmpdir=/tmp

starrynight inventory gen -d /home/ank/workspace/hub/broad/starrynight/scratch/starrynight_example_input -o /home/ank/workspace/hub/broad/starrynight/scratch/starrynight_example_output/inventory
Writing inventory: 100%|██████████| 1/1 [00:00<00:00, 6087.52it/s]









Writing inventory: 100%|███

## Configure the generate index module
This module is special and doesn't require an experiment for configuration

In [7]:
gen_ind_mod = GenIndexModule.from_config(data_config)
exec_backend = SnakeMakeBackend(
    gen_ind_mod.pipe, backend_config, exec_runs / "run002", exec_mounts
)
run = exec_backend.run()

Assuming unrestricted shared filesystem usage.
host: GPFDA-11A
Building DAG of jobs...
Using shell: /nix/store/8vpg72ik2kgxfj05lc56hkqrdrfl8xi9-bash-5.2p37/bin/bash
Provided cores: 10
Rules claiming more threads will be scaled down.
Singularity containers: ignored
Job stats:
job               count
--------------  -------
all                   1
generate_index        1
total                 2

Select jobs to execute...
Execute 1 jobs...

[Thu Apr 17 14:07:52 2025]
localrule generate_index:
    input: /home/ank/workspace/hub/broad/starrynight/scratch/starrynight_example_output/inventory/inventory.parquet
    output: /home/ank/workspace/hub/broad/starrynight/scratch/starrynight_example_output/index/index.parquet
    jobid: 1
    reason: Forced execution
    resources: tmpdir=/tmp

starrynight index gen -i /home/ank/workspace/hub/broad/starrynight/scratch/starrynight_example_output/inventory/inventory.parquet -o /home/ank/workspace/hub/broad/starrynight/scratch/starrynight_example_output/

## Configure the experiment with the generated index

In [7]:
index_path = workspace_path / "index/index.parquet"
pcp_exp_init = PCPGenericInitConfig(
    barcode_csv_path=dataset_path / "workspace/metadata/barcode.csv",
    cp_acquisition_order=AcquisitionOrderType.SNAKE,
    cp_img_frame_type=ImageFrameType.ROUND,
    cp_img_overlap_pct=10,
    sbs_acquisition_order=AcquisitionOrderType.SNAKE,
    sbs_img_frame_type=ImageFrameType.ROUND,
    sbs_img_overlap_pct=10,
    cp_nuclei_channel="DAPI",
    cp_cell_channel="PhalloAF750",
    cp_mito_channel="ZO1AF488",
    sbs_nuclei_channel="DAPI",
    sbs_cell_channel="PhalloAF750",
    sbs_mito_channel="ZO1AF488",
)
pcp_experiment = PCPGeneric.from_index(index_path, pcp_exp_init.model_dump())

## Configure the following modules with the experiment

------------------------------------------------------------------

## Create and exec pipeline

In [8]:
# Create pipeline
modules, pcp_generic_pipeline = create_pcp_generic_pipeline(
    data_config, pcp_experiment
)

In [8]:
modules

[CPCalcIllumGenLoadDataModule(spec=Container(inputs=[TypeInput(name='index_path', type='files', description='Path to the index.', optional=False, path='/home/ank/workspace/hub/broad/starrynight/scratch/starrynight_example_output/index/index.parquet', format=None, collection=None, subtype=None, z=None, t=None, tiled=None, pyramidal=None), TypeInput(name='path_mask', type='file', description='Path prefix mask to use.', optional=True, path=None, format=None, collection=None, subtype=None, z=None, t=None, tiled=None, pyramidal=None)], outputs=[TypeOutput(name='calc_illum_loaddata', type='files', description='Generated Illum calc loaddata files', optional=False, path='/home/ank/workspace/hub/broad/starrynight/scratch/starrynight_example_output/cellprofiler/loaddata/cp/illum_calc', format=None, collection=None, subtype=None, z=None, t=None, tiled=None, pyramidal=None), TypeOutput(name='loaddata_notebook', type='notebook', description='Notebook for inspecting load data files', optional=False,

In [8]:
pcp_generic_pipeline.compile()
pcp_generic_pipeline.pipeline.nodes

NodeView((cp_calc_illum_gen_loaddata(CS), calc_illum_gen_cppipe(CS), cp_calc_illum_invoke_cp(CS), cp_apply_illum_gen_loaddata(CS), cp_apply_illum_gen_cppipe(CS), cp_apply_illum_invoke_cp(CS), cp_segchek_gen_loaddata(CS), cp_segcheck_gen_cppipe(CS), cp_segcheck_invoke_cp(CS), Scatter, Gather, sbs_calc_illum_gen_loaddata(CS), sbs_calc_illum_gen_cppipe(CS), sbs_calc_illum_invoke_cp(CS), sbs_apply_illum_gen_loaddata(CS), sbs_apply_illum_gen_cppipe(CS), sbs_apply_illum_invoke_cp(CS), sbs_preprocess_gen_loaddata(CS), sbs_preprocess_gen_cppipe(CS), sbs_preprocess_invoke_cp(CS), analysis_gen_loaddata(CS), analysis_gen_cppipe(CS), analysis_invoke_cp(CS)))

In [9]:
# Exec pipeline
exec_backend = SnakeMakeBackend(
    pcp_generic_pipeline,
    backend_config,
    exec_runs / "run003",
    exec_mounts,
)
exec_backend.compile()

Scatter
Gather


In [10]:
run = exec_backend.run()

Scatter
Gather


Assuming unrestricted shared filesystem usage.
host: GPFDA-11A
Building DAG of jobs...
Using shell: /nix/store/8vpg72ik2kgxfj05lc56hkqrdrfl8xi9-bash-5.2p37/bin/bash
Provided cores: 10
Rules claiming more threads will be scaled down.
Singularity containers: ignored
Job stats:
job                             count
----------------------------  -------
all                                 1
analysis_gen_cppipe                 1
analysis_gen_loaddata               1
analysis_invoke_cp                  1
calc_illum_gen_cppipe               1
cp_apply_illum_gen_cppipe           1
cp_apply_illum_gen_loaddata         1
cp_apply_illum_invoke_cp            1
cp_calc_illum_gen_loaddata          1
cp_calc_illum_invoke_cp             1
cp_segcheck_gen_cppipe              1
cp_segcheck_invoke_cp               1
cp_segchek_gen_loaddata             1
sbs_apply_illum_gen_cppipe          1
sbs_apply_illum_gen_loaddata        1
sbs_apply_illum_invoke_cp           1
sbs_calc_illum_gen_cppipe           1
sb

pipeline_exception


[Thu Apr 17 17:39:07 2025]
Error in rule sbs_apply_illum_invoke_cp:
    jobid: 15
    input: /home/ank/workspace/hub/broad/starrynight/scratch/starrynight_example_output/cellprofiler/cppipe/sbs/illum_apply/completed.txt, /home/ank/workspace/hub/broad/starrynight/scratch/starrynight_example_output/cellprofiler/loaddata/sbs/illum_apply/completed.txt, /home/ank/workspace/hub/broad/starrynight/scratch/starrynight_example_output/illum/sbs/illum_calc/completed.txt
    output: /home/ank/workspace/hub/broad/starrynight/scratch/starrynight_example_output/illum/sbs/illum_apply/completed.txt
    shell:
        starrynight cp -p /home/ank/workspace/hub/broad/starrynight/scratch/starrynight_example_output/cellprofiler/cppipe/sbs/illum_apply/illum_apply_sbs.cppipe -l /home/ank/workspace/hub/broad/starrynight/scratch/starrynight_example_output/cellprofiler/loaddata/sbs/illum_apply -o /home/ank/workspace/hub/broad/starrynight/scratch/starrynight_example_output/illum/sbs/illum_apply --sbs
        (one 